In [106]:
import pandas as pd

base_path = 'ml-100k/'

# 1. Load ratings (u.data: tab-separated userId | movieId | rating | timestamp)
ratings_cols = ['userId','movieId','rating','timestamp']
ratings = pd.read_csv(
    f'{base_path}u.data', 
    sep='\t', 
    names=ratings_cols, 
    engine='python'
)

# 2. Load movies (u.item: pipe-separated; first 5 cols + 19 genre flags)
item_cols = ['movieId','title','release_date','video_release_date','IMDb_URL']
genre_cols = [
    'unknown','Action','Adventure','Animation',"Children's",'Comedy','Crime',
    'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
    'Romance','Sci-Fi','Thriller','War','Western'
]
movies = pd.read_csv(
    f'{base_path}u.item',
    sep='|',
    names=item_cols + genre_cols,
    encoding='latin-1',
    engine='python'
)

# 3. Quick sanity-check
print(ratings.head())
print(movies.head())


   userId  movieId  rating  timestamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596
   movieId              title release_date  video_release_date  \
0        1   Toy Story (1995)  01-Jan-1995                 NaN   
1        2   GoldenEye (1995)  01-Jan-1995                 NaN   
2        3  Four Rooms (1995)  01-Jan-1995                 NaN   
3        4  Get Shorty (1995)  01-Jan-1995                 NaN   
4        5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://

In [ ]:
# Jul 18

# mean rating of movies dataframe, converted to integer as to be a categorical feature
mean_ratings = ratings.groupby(ratings.movieId).mean()[['rating']].astype(int) 

# merge movies dataframe on mean_ratings dataframe by movieId
# final_df contains genres as features (True or False) and Ratings as a feature (1-5)
final_df = pd.merge(mean_ratings, movies.set_index('movieId'), left_index= True, right_index=True)

def p_X_given_y(watched_df, row): #returns P(x_1 | y), P(x_2 | y), P(x_3 | y)...
    row = row.drop(['title', 'release_date', 'video_release_date', 'IMDb_URL']) #drop non-features
    matches = (watched_df[row.index] == row.values).sum() #count(X, y)
    return (1 + matches) / (1 + len(watched_df))  #P(X | y) = count(X, y) / count(y), with laplace smoothing


def p_y_given_X(row, watched_df, final_df): #P(y | X) ∝ P(Y) * ∏ P(X | y)
    p_y = len(watched_df) / len(final_df) #P(Y) = count(Y) / count(All)
    p_X_given_y_vector = p_X_given_y(watched_df, row) #P(x_1 | y), P(x_2 | y), P(x_3 | y)...
    return p_y * p_X_given_y_vector.prod() #P(y | X) ∝ P(Y) * ∏ P(X | y)


# compute P(y|X) for all unwatched movies
# accepts as input the whole dataframe of all movies(final_df), and the dataframe of all watched movies by the user(watched_df)
def naive_bayes(final_df, watched_df):
    unwatched_df = pd.concat([final_df, watched_df, watched_df]).drop_duplicates(keep=False) #unwatched_movies df
    unwatched_df['p_y_given_X'] = unwatched_df.apply(p_y_given_X, axis = 1, args = (watched_df, final_df)) # P(y | X) for all unwatched movies
    return unwatched_df.sort_values(by = 'p_y_given_X', ascending = False)['title'].head(10) # returns 10 movies with highest probability


naive_bayes(final_df, final_df.sample(frac=0.2))  # Randomly select 20% of the DataFrame as watched

movieId
1682            Scream of Stone (Schrei aus Stein) (1991)
1171                                    Wild Reeds (1994)
1147                                     My Family (1995)
1148                                     Tom & Viv (1994)
1149                                     Walkabout (1971)
1150                                    Last Dance (1996)
1630    Silence of the Palace, The (Saimt el Qusur) (1...
1168                                 Little Buddha (1993)
1640                               Eighth Day, The (1996)
1205                             Secret Agent, The (1996)
Name: title, dtype: object